# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1: for first table 'songs_by_user_session' , since we are looking for the specific session id and items in session, we will start with that and make a composite key of the 'sessionId' & 'itemsInSession', to filter by that columns and it will be enough to make the row key unique.

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# Create first table database to get the first query result
try:
    session.execute("""
                       CREATE TABLE IF NOT EXISTS songs_by_user_session(
                                                          sessionId      INT,
                                                          itemInSession  INT,
                                                          artist         TEXT,
                                                          song_title     TEXT,
                                                          song_length    FLOAT,
                                                          PRIMARY KEY(sessionId,itemInSession))
                                                          """)
except Exception as e:
    print(e)

In [10]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the query variable
        query = "INSERT INTO songs_by_user_session (sessionId,itemInSession,artist,song_title,song_length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query,(int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

In [11]:
## Add in the SELECT statement to verify the data was entered into the table
# Use Select to verify the data that have been inserted into the table
query = " SELECT artist,song_title,song_length \
          FROM songs_by_user_session \
          WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist,row.song_title,row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: for second table 'artists_songs_by_user_session', since we are looking for the specific user id and  session id, we will start with that and make a composite key of the 'user_Id' & 'sessionId', and add 'itemsInSession' as a cluster  column to sort the data by it.

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# Create second table database to get the second query result
try:
    session.execute("""
                       CREATE TABLE IF NOT EXISTS artists_songs_by_user_session(
                                                          user_id        INT,
                                                          sessionId      INT,
                                                          itemInSession  INT,
                                                          artist         TEXT,
                                                          song_title     TEXT,
                                                          first_name     TEXT,
                                                          last_name      TEXT,
                                                          PRIMARY KEY((user_id,sessionId),itemInSession))
                                                          """)
except Exeption as e:
    print(e)
                    

In [13]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file,encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        # insert statment into the query variable
        query = "INSERT INTO artists_songs_by_user_session(user_id,sessionId,itemInSession,\
                                                         artist,song_title,first_name,last_name)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        # assigned each column in the insert statement
        session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [14]:
# use SELECT statment to verify the data was entered into the artist_song_by_user_session
query = "SELECT artist,song_title,first_name,last_name \
         FROM artists_songs_by_user_session \
         WHERE user_id = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist,row.song_title,row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: for third table 'users_by_song', since we are looking for the specific song title, we will start with that and make a primary key of the 'song_title', and add 'user_id' as a cluster  column to make the data unique. that should be enough to make the row key unique.

In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Create third table database to get the third query result 
try:
    session.execute("""
                       CREATE TABLE IF NOT EXISTS users_by_song(
                                                          song_title TEXT,
                                                          user_id    INT,
                                                          first_name TEXT,
                                                          last_name  TEXT,
                                                          PRIMARY KEY(song_title,user_id))
                                                          """)
except Execption as e:
    print(e)
                    

In [16]:
# set up CSV file
file = 'event_datafile_new.csv'

with open(file,encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insert statment into the query variable
        query = "INSERT INTO users_by_song(song_title,user_id,first_name,last_name)"
        query = query + "VALUES(%s,%s,%s,%s)"
        # assigned each column in the insert statement
        session.execute(query,(line[9],int(line[10]),line[1],line[4]))

In [17]:
# Use SELECT statment to verify the data was entered in the table_3
query = "SELECT first_name,last_name \
         FROM users_by_song \
         WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.first_name,row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions
# drop first table
try:
    session.execute("DROP TABLE songs_by_user_session")
except Exception as e:
    print(e)

In [19]:
# drop second table
try:
    session.execute("DROP TABLE artists_songs_by_user_session")
except Exception as e:
    print(e)

In [20]:
# drop third table
try:
    session.execute("DROP TABLE users_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()